In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install odfpy
!pip install tqdm
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3,390 kB]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:13 https://developer.download.nvidia.com/compute/machi

In [ ]:
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm


In [ ]:
#Import Dataset
from google.colab import files
df_src = pd.read_csv('https://raw.githubusercontent.com/dan-nang/lpse_data/main/lpse_url.csv')


In [ ]:
# get urls
df = df_src['url']
df_cek = df.iloc[0:100]+"/lelang"
df.tail()

677        http://110.138.137.229/eproc4
678          http://lpse.ut.ac.id/eproc4
679    https://lpse.jakarta.go.id/eproc4
680        http://eproc.kbn.co.id/eproc4
681      https://lpse.pelni.co.id/eproc4
Name: url, dtype: object

In [ ]:
driver = webdriver.Chrome('chromedriver',options=options)

# declare list variables
link_lpse_list=[]
kode_list = []
nama_paket_list = []
link_paket_list=[]
detail_paket_list=[]
status_list=[]
nilai_kontrak_list=[]
instansi_list=[]
link_tahapan_list=[]
tahapan_list=[]
hps_list=[]
err_link = []

In [ ]:
# set Urls based on index in dataset
i=606
last=607

# Looping to put data in variables
for i in tqdm(range(i, last)):
  try:
    # url from dataset add /lelang
    lspe_link = "{}".format(df.iloc[i]+"/lelang")

    driver.set_window_size(1920,1080)

    # get website
    driver.get(lspe_link)
    # waiting to website load succesfully
    time.sleep(5)

    # source of website #1
    data = BeautifulSoup(driver.page_source, 'lxml')

    # set items per page to all
    # driver.find_element(By.NAME,"tbllelang_length") #set items per page to all
    try:
      val_temp = data.find('option', {'value' : '-1'}).text
      driver.find_element(By.NAME,"tbllelang_length").send_keys('semua')
    except:
      driver.find_element(By.NAME,"tbllelang_length").send_keys('100')
    # waiting to website load succesfully
    time.sleep(20)

     # source of website #2
    data = BeautifulSoup(driver.page_source, 'lxml')
    
    # looping to get main data from one url
    for area in data.find_all('tr', {'class': ['odd','even']}):

      link_lpse_list.append(lspe_link)

      # find kode to be scripted
      kode = area.find('td', class_="sorting_1").get_text()
      # append scripting data to variable
      kode_list.append(kode)

      nama_paket = area.find('a').contents[0]
      nama_paket_list.append(nama_paket)

      link_paket = area.find('a', href=True)['href']
      link_paket_list.append(link_paket)

      detail_paket = area.find_next('p').find_next('p').text
      detail_paket_list.append(detail_paket)

      status = [e.text for e in area.find_all('span')]
      status_list.append(status)

      nilai_kontrak = area.find_next('p').find_next('p').find_next('p').text.replace("Nilai Kontrak : ", "")
      nilai_kontrak_list.append(nilai_kontrak)

      instansi = area.find('td').find_next('td').find_next('td').text
      instansi_list.append(instansi)

      link_tahapan = area.find('a').find_next('a', href=True)['href']
      link_tahapan_list.append(link_tahapan)

      tahapan = area.find('a').find_next('a', href=True).text
      tahapan_list.append(tahapan)

      hps = area.find('td').find_next('td').find_next('td').find_next('td').find_next('td').text
      hps_list.append(hps)
        
    # next index url
    i= i+1
  except:
    # next index url
    i= i+1
    err_link.append(lspe_link)
    pass

100%|██████████| 1/1 [00:28<00:00, 28.65s/it]


In [ ]:
tender_list =[]
spse_list =[]
info_list = []
for sublist in status_list:
  tmp_tender=[]
  tmp_spse=[]
  tmp_info=[]
  for e in sublist:
    if "Tender"in e:
      tmp_tender.append(e)
    elif "spse" in e:
      tmp_spse.append(e)
    else:
      tmp_info.append(e)
  tender_list.append(tmp_tender)
  spse_list.append(tmp_spse)
  info_list.append(tmp_info)

In [ ]:
# add data to dictionary
# to_dict_lengkap = {"Kode":kode_list,"Url_LPSE":link_lpse_list, "Link Paket":link_paket_list, "Nama Paket":nama_paket_list, "Detail Paket":detail_paket_list,
#                     "Tender":tender_list,"SPSE":spse_list,"Info":info_list, 
#                     "Status":status_list,
#                     "Nilai Kontrak":nilai_kontrak_list, "K/L/PD":instansi_list,
#                     "Link Tahapan":link_tahapan_list, "Tahapan":tahapan_list, "HPS":hps_list}

to_dict= {"Kode":kode_list,
          "URL LPSE":link_lpse_list,
          "Nama Paket":nama_paket_list, "Detail Paket":detail_paket_list,
          "Tender":tender_list,"SPSE":spse_list,"Info":info_list, 
          "Nilai Kontrak":nilai_kontrak_list, "K/L/PD":instansi_list,
          "Tahapan":tahapan_list, 
          "HPS":hps_list}
      
err_urls = {"Error url": err_link}

In [ ]:
import pandas as pd

# add to dataframe and transform data
df_hasil = pd.DataFrame.from_dict(to_dict, orient='index')
err = pd.DataFrame.from_dict(err_urls, orient='index')
df_transform = df_hasil.T
err_url = err.T

# count row data
df_transform[df_transform.columns[0]].count()

100

In [ ]:
# count error links
err_url[err_url.columns[0]].count()

0

In [ ]:
# err_url.iloc[11:21]
err_url['Error url']

Series([], Name: Error url, dtype: float64)

In [ ]:
df_transform

,Kode,URL LPSE,Nama Paket,Detail Paket,Tender,SPSE,Info,Nilai Kontrak,K/L/PD,Tahapan,HPS
0,1684303,https://lpse.ugm.ac.id/eproc4/lelang,Pengadaan Peralatan Laboratorium Departemen TP...,Pengadaan Barang - TA 2022 - Tender Cepat - Pa...,[],[spse 4.3],[],Nilai Kontrak belum dibuat,Universitas Gadjah Mada,Tender Sudah Selesai,"632,6 Jt"
1,1683303,https://lpse.ugm.ac.id/eproc4/lelang,Pengadaan Peralatan Laboratorium Departemen TP...,Pengadaan Barang - TA 2022 - Tender Cepat - Pa...,[],[spse 4.3],[],Nilai Kontrak belum dibuat,Universitas Gadjah Mada,Tender Sudah Selesai,"748,5 Jt"
2,1682303,https://lpse.ugm.ac.id/eproc4/lelang,Pengadaan Almari Asam untuk Laboratorium Tekno...,Pengadaan Barang - TA 2022 - Tender Cepat - Pa...,[],[spse 4.3],[],Nilai Kontrak belum dibuat,Universitas Gadjah Mada,Tender Sudah Selesai,400 Jt
3,1681303,https://lpse.ugm.ac.id/eproc4/lelang,Pengadaan Peralatan Komputer IT Fakultas Geogr...,Pengadaan Barang - TA 2022 - Tender - Pascakua...,[],[spse 4.3],[],"Rp 241.914.993,96",Universitas Gadjah Mada,Tender Sudah Selesai,280 Jt
4,1680303,https://lpse.ugm.ac.id/eproc4/lelang,Sewa Layanan Google Education Suite,Jasa Lainnya - TA 2022 - Tender Cepat - Pascak...,[],[spse 4.3],[],"Rp 4.379.550.000,00",Universitas Gadjah Mada,Tender Sudah Selesai,"4,7 M"
...,...,...,...,...,...,...,...,...,...,...,...
95,1554303,https://lpse.ugm.ac.id/eproc4/lelang,Pengadaan Komputer Tablet Paket 2 DPP UGM Tahu...,Pengadaan Barang - TA 2021 - Tender Cepat - Pa...,[],[spse 4.3],[],"Rp 2.666.400.000,00",Universitas Gadjah Mada,Tender Sudah Selesai,3 M
96,1553303,https://lpse.ugm.ac.id/eproc4/lelang,Pengadaan Komputer Tablet Paket 1 DPP UGM Tahu...,Pengadaan Barang - TA 2021 - Tender Cepat - Pa...,[],[spse 4.3],[],"Rp 3.013.862.500,00",Universitas Gadjah Mada,Tender Sudah Selesai,"3,1 M"
97,1551303,https://lpse.ugm.ac.id/eproc4/lelang,Pengadaan Peralatan Laboratorium DTETI FT UGM,Pengadaan Barang - TA 2021 - Tender Cepat - Pa...,[],[spse 4.3],[],"Rp 970.126.000,00",Universitas Gadjah Mada,Tender Sudah Selesai,1 M
98,1550303,https://lpse.ugm.ac.id/eproc4/lelang,PEMBANGUNAN KANDANG HEWAN RSH PROF. SOEPARWI U...,Pekerjaan Konstruksi - TA 2021 - Tender - Pasc...,[],[spse 4.3],[],"Rp 3.089.652.677,21",Universitas Gadjah Mada,Tender Sudah Selesai,"3,4 M"


In [ ]:
# Replace empty list with empty string
df_transform = df_transform.mask(df_transform.applymap(str).eq('[]'))
df_transform = df_transform.fillna('')

# Hapus kurung siku
df_transform = df_transform.apply(lambda x: x.explode().astype(str).groupby(level=0).agg(", ".join)) 

# Split detail paket dengan delimiter (-)
df_transform[["Jenis Pengadaan", "Tahun Anggaran", "Jenis Tender", "Pascakualifikasi"]] = df_transform["Detail Paket"].str.split("-", expand=True)

# Merge kolom Jenis Tender dan Pascakualifikasi sebagai satu kolom Metode Pengadaan
df_transform["Metode Pengadaan"] = df_transform["Jenis Tender"].str.cat(df_transform["Pascakualifikasi"].astype(str), sep="-")

# Hapus Kolom
df_transform.drop(columns =['Detail Paket', 'Jenis Tender', 'Pascakualifikasi'], inplace =True)

In [ ]:
# Pindah kolom 
kolom_fix = ['Kode', 'URL LPSE',
 'Nama Paket',
 'Jenis Pengadaan',
 'Tahun Anggaran',
 'Metode Pengadaan',
 'Tender',
 'SPSE',
 'Info',
 'Nilai Kontrak',
 'K/L/PD',
 'Tahapan',
 'HPS']
 
df_transform = df_transform[kolom_fix]

In [ ]:
df_transform

,Kode,URL LPSE,Nama Paket,Jenis Pengadaan,Tahun Anggaran,Metode Pengadaan,Tender,SPSE,Info,Nilai Kontrak,K/L/PD,Tahapan,HPS
0,1684303,https://lpse.ugm.ac.id/eproc4/lelang,Pengadaan Peralatan Laboratorium Departemen TP...,Pengadaan Barang,TA 2022,Tender Cepat - Pascakualifikasi Satu File Har...,,spse 4.3,,Nilai Kontrak belum dibuat,Universitas Gadjah Mada,Tender Sudah Selesai,"632,6 Jt"
1,1683303,https://lpse.ugm.ac.id/eproc4/lelang,Pengadaan Peralatan Laboratorium Departemen TP...,Pengadaan Barang,TA 2022,Tender Cepat - Pascakualifikasi Satu File Har...,,spse 4.3,,Nilai Kontrak belum dibuat,Universitas Gadjah Mada,Tender Sudah Selesai,"748,5 Jt"
2,1682303,https://lpse.ugm.ac.id/eproc4/lelang,Pengadaan Almari Asam untuk Laboratorium Tekno...,Pengadaan Barang,TA 2022,Tender Cepat - Pascakualifikasi Satu File Har...,,spse 4.3,,Nilai Kontrak belum dibuat,Universitas Gadjah Mada,Tender Sudah Selesai,400 Jt
3,1681303,https://lpse.ugm.ac.id/eproc4/lelang,Pengadaan Peralatan Komputer IT Fakultas Geogr...,Pengadaan Barang,TA 2022,Tender - Pascakualifikasi Satu File Harga Ter...,,spse 4.3,,"Rp 241.914.993,96",Universitas Gadjah Mada,Tender Sudah Selesai,280 Jt
4,1680303,https://lpse.ugm.ac.id/eproc4/lelang,Sewa Layanan Google Education Suite,Jasa Lainnya,TA 2022,Tender Cepat - Pascakualifikasi Satu File Har...,,spse 4.3,,"Rp 4.379.550.000,00",Universitas Gadjah Mada,Tender Sudah Selesai,"4,7 M"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1554303,https://lpse.ugm.ac.id/eproc4/lelang,Pengadaan Komputer Tablet Paket 2 DPP UGM Tahu...,Pengadaan Barang,TA 2021,Tender Cepat - Pascakualifikasi Satu File Har...,,spse 4.3,,"Rp 2.666.400.000,00",Universitas Gadjah Mada,Tender Sudah Selesai,3 M
96,1553303,https://lpse.ugm.ac.id/eproc4/lelang,Pengadaan Komputer Tablet Paket 1 DPP UGM Tahu...,Pengadaan Barang,TA 2021,Tender Cepat - Pascakualifikasi Satu File Har...,,spse 4.3,,"Rp 3.013.862.500,00",Universitas Gadjah Mada,Tender Sudah Selesai,"3,1 M"
97,1551303,https://lpse.ugm.ac.id/eproc4/lelang,Pengadaan Peralatan Laboratorium DTETI FT UGM,Pengadaan Barang,TA 2021,Tender Cepat - Pascakualifikasi Satu File Har...,,spse 4.3,,"Rp 970.126.000,00",Universitas Gadjah Mada,Tender Sudah Selesai,1 M
98,1550303,https://lpse.ugm.ac.id/eproc4/lelang,PEMBANGUNAN KANDANG HEWAN RSH PROF. SOEPARWI U...,Pekerjaan Konstruksi,TA 2021,Tender - Pascakualifikasi Dua File Sistem Nilai,,spse 4.3,,"Rp 3.089.652.677,21",Universitas Gadjah Mada,Tender Sudah Selesai,"3,4 M"


In [ ]:
# df_transform.to_excel("lpse_400_500.xlsx",index = False)


In [ ]:
# err_url.to_excel("err_lpse_571_600.xlsx",index = False)